# ♎ IC Passives

In this notebook, we'll cover passive components such as resistors and capacitors in IC design. We'll explore how these components are made and operate, both within the silicon substrate and the metallization layers added during the IC manufacturing process.

🎥 Video Insight:

<iframe width="560" height="315" src="https://www.youtube.com/embed/nhPX8IFYk9Y?si=-4F-BuWoEnhkpoKp" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

Now, let's take a look at the GF180 MCU PDK to see how these components manifest themselves!

### Polysilicon/Metal Resistors

![Strip Resistor](img/strip_resistor.png)

#### Resistivity

Resistors are often crafted from silicide-doped polysilicon or metal interconnects. Their behavior is majorly governed by the resistance per unit ratio parameter, $R_{\square}$, with the actual resistance determined by:
$$R = \dfrac{R_{\square}L}{W}$$

The sheet resistance of these materials isn't listed in the GF180 PDK documentation, which is perfect for us as we can now deduce them in simulations here for our main exercise:

Deduce $R_{\square}$ for the following devices:

| PDK Name         | Hdl21 Name      | Description                                                      | $R_{\square}$ |
|------------------|-----------------|------------------------------------------------------------------|-------------------|
| npolyf_u         | NPOLYF_U        | 3-T Model for unsalicided N+ poly resistor                      | ?                 |
| ppolyf_u         | PPOLYF_U        | 3-T Model for unsalicided P+ poly resistor                      | ?                 |
| npolyf_s         | NPOLYF_S        | 3-T Model for salicided N+ poly resistor                        | ?                 |
| ppolyf_s         | PPOLYF_S        | 3-T Model for salicided P+ poly resistor                        | ?                 |
| ppolyf_u_1k      | PPOLYF_U_1K     | 3-T Model for 1k ohm hi sheet rho P poly resistor (3.3V)       | ?                 |
| ppolyf_u_2k      | PPOLYF_U_2K     | 3-T Model for 2k ohm hi sheet rho P poly resistor (3.3V)       | ?                 |
| ppolyf_u_1k_6p0  | PPOLYF_U_1K_6P0 | 3-T Model for 1k ohm hi Sheet rho P poly resistor (6.0V)       | ?                 |
| ppolyf_u_2k_6p0  | PPOLYF_U_2K_6P0 | 3-T Model for 2k ohm hi sheet rho P poly resistor (6.0V)       | ?                 |
| ppolyf_u_3k      | PPOLYF_U_3K     | 3-T Model for 3k ohm hi sheet rho P poly resistor (3.3V)       | ?                 |
| rm1              | RM1             | 2-T Model for metal 1 resistor                                  | ?                 |
| rm2              | RM2             | 2-T Model for metal 2 resistor                                  | ?                 |
| rm3              | RM3             | 2-T Model for metal 3 resistor                                  | ?                 |
| tm6k             | TM6K            | 2-T Model for top metal 6k resistor                             | ?                 |
| tm9k             | TM9K            | 2-T Model for top metal 9k resistor                             | ?                 |
| tm11k            | TM11K           | 2-T Model for top metal 11k resistor                            | ?                 |
| tm30k            | TM30K           | 2-T Model for top metal 30k resistor                            | ?                 |

In [ ]:
import hdl21 as h
from hdl21.prefix import u
import gf180_hdl21 as g
import gf180_hdl21.primitives as p
import vlsir.spice_pb2 as vsp

@h.sim.sim
class ResSim:

    @h.module
    class Tb:

        VSS = h.Port()
        VDD = h.Signal()

        vdd = h.DcVoltageSource(dc=1)(p=VDD,n=VSS)

        # Insert whatever resistor you'd like
        # Two terminals are p and n
        # Three terminals are p, n, and b
        res = p.NPOLYF_S(r_width=0, r_length=0)(p=VDD,n=VSS,b=VSS)

    op = h.sim.Op()
    inc1 = g.install.include_design()
    inc2 = g.install.include_resistors()

rvs = ResSim.run()
result = rvs[vsp.sim_data.AnalysisType.OP]

print(result)

#### Parasitic Capacitance

While the nominal resistivity of our resistors is probably the most interesting parameter to us - every integrated device is also secretly a capacitor and it's important for us to be able to detect and know the approximate parameters of this parasitic capacitance. The parasitic capacitance of a poly resistor is that of a sheet capacitor, with the poly acting as a plate on the top and the silicon beneath acting as the plate on the bottom, ie.

$$C = C_0WL$$

So, we can measure the parasitic capacitance by looking at frequency response of resistor to a signal across the two "ordinary" terminals of the resistor and ground to get results. Below we've provided the testbench to do this AC analysis on - see if you can complete the exercises below:

##### 🧠 Exercises

1. Simulate a circuit with a poly resistor and an AC source. Measure the frequency response and determine the point the parasitic capacitance
2. Research and discuss real-world scenarios where parasitic capacitance in poly resistors has caused issues in circuit performance. How were these issues mitigated?

In [ ]:
import numpy as np

def measure_capacitance(device, width, length):
    
    @h.sim.sim
    class CapacitanceSim:

        @h.module
        class Tb:

            VSS = h.Port()
            VDD = h.Signal()

            vdd = h.DcVoltageSource(ac=1)(p=VDD, n=VSS)
            res = device(rwidth=width,r_length=length)(p=VDD, n=VDD, b=VSS)

        # Simulation Controls
        ac = h.sim.Ac(sweep=h.sim.LogSweep(1e1, 1e10, 10))
        inc1 = g.install.include_design()
        inc2 = g.install.include_resistors()

    # Run the simulation
    opts = vsp.SimOptions(
        simulator=vsp.SupportedSimulators.NGSPICE,
        fmt=vsp.ResultFormat.SIM_DATA,
        rundir="./scratch",
    )
    rv = CapacitanceSim.run(opts)
    result = rv[vsp.sim_data.AnalysisType.AC]

    # Write some mathematics to recover the parasitic capacitance

### Semiconductor Resistors

![Semiconductor Resistors](img/semiconductor_resistor.png)

##### Resistivity

The resistivity of a semiconductor resistors is very similar to sheet resistors with resistivity being determined by dimensions and a sheet resistivity, we show which types of semiconductors are available to us in the GF180 PDK:

| PDK Name  | Hdl21 Name | Description                                          | $R_{\square}$ |
|-----------|------------|------------------------------------------------------|-------------------|
| nplus_u   | NPLUS_U    | 3-T Model for unsalicided N+ diffusion resistor      | ?                 |
| pplus_u   | PPLUS_U    | 3-T Model for unsalicided P+ diffusion resistor      | ?                 |
| nplus_s   | NPLUS_S    | 3-T Model for salicided N+ diffusion resistor        | ?                 |
| pplus_s   | PPLUS_S    | 3-T Model for salicided P+ diffusion resistor        | ?                 |
| nwell     | NWELL      | 3-T Model for Nwell resistor                         | ?                 |

##### 🧠 Exercises

1. Determine $R_\square$ of these resistors again, how do they compare to the poly resistors?
2. Repeat the AC analysis above to determine the parasitic capacitance, how do they vary?
3. Simulate both poly and semiconductor resistors at different temperatures, what happens?

In [ ]:
# Implement your solution here!

### MOS Capacitors

![Mos Cap Structure](img/semiconductor_capacitor.png)

The simplest design for a capacitor in silicon is a MOSFET itself, these are provided in the GF180 PDK as:

| PDK Name        | Hdl21 Name       | Description                                                   | C   |
|-----------------|------------------|---------------------------------------------------------------|-----|
| cap_pmos_03v3   | PMOS_3p3V        | Model for nominal IO 3.3V inversion-mode PMOS capacitor       | ?   |
| cap_nmos_06v0   | NMOS_6p0V        | Model for nominal IO 6V inversion -mode NMOS capacitor        | ?   |
| cap_pmos_06v0   | PMOS_6p0V        | Model for nominal IO 6V inversion -mode PMOS capacitor        | ?   |
| cap_nmos_03v3   | NMOS_3p3V        | Model for nominal IO 3.3V inversion-mode NMOS capacitor       | ?   |
| cap_nmos_03v3_b | NMOS_Nwell_3p3V  | Model for nominal IO 3.3V NMOS in Nwell capacitor             | ?   |
| cap_pmos_03v3_b | PMOS_Pwell_3p3V  | Model for nominal IO 3.3V PMOS in Pwell capacitor             | ?   |
| cap_nmos_06v0_b | NMOS_Nwell_6p0V  | Model for nominal IO 6V NMOS in Nwell capacitor               | ?   |
| cap_pmos_06v0_b | PMOS_Pwell_6p0V  | Model for nominal IO 6V PMOS in Pwell capacitor               | ?   |

##### 🧠 Exercises

1. Determine the saturated capacitance of some of these capacitors
2. Characterize the non-linear capacitance of the capacitors as you vary $V_{GS}$
3. Determine any parasitic resistances that could be found in these models.

In [ ]:
# Simulate your solution here!

@h.sim.sim
class CapacitanceSim:

    @h.module
    class Tb:

        VSS = h.Port()
        VDD = h.Signal()

        vdd = h.DcVoltageSource(ac=1)(p=VDD, n=VSS)
        res = p.NPLUS_S(c_width=0,c_length=0)(p=VDD, n=VDD, b=VSS)

    # Simulation Controls
    ac = h.sim.Ac(sweep=h.sim.LogSweep(1e1, 1e10, 10))
    inc1 = g.install.include_design()
    inc2 = g.install.include_moscaps(h.pdk.Corner.TYP)

# Run the simulation
opts = vsp.SimOptions(
    simulator=vsp.SupportedSimulators.NGSPICE,
    fmt=vsp.ResultFormat.SIM_DATA,
    rundir="./scratch",
)
rv = CapacitanceSim.run(opts)
result = rv[vsp.sim_data.AnalysisType.AC]

### MiM Capacitors

![Metal-on-metal Capacitor](img/metal_on_metal_capacitor.png)

For more linear resistors, we're better off using more conventional capacitor designs in the form of "metal-insulator-metal" capacitors or MiM capacitors, these are given in the GF180 PDK, by:

| PDK Name        | Hdl21 Name  | Description | C ($F/\mu m^2$)   |
|-----------------|-------------|-------------|-----|
| cap_mim_1f5fF   | MIM_1p5fF   | ?           | ?   |
| cap_mim_1f0fF   | MIM_1p0fF   | ?           | ?   |
| cap_mim_2f0fF   | MIM_2p0fF   | ?           | ?   |

Unfortunately, there are no interleaved metal capacitors that have been characterized for the GF180 process, however, if you wanted to tapeout and characterize such capacitors yourself - this might make for a great first analog project!

##### 🧠 Exercises

1. Find the sheet capacitance of the MiM caps listed above
2. Can you isolate any parasitics from the model?

In [ ]:
# Simulate your solution here!